### Dependencies

In [ ]:
!pip install scikit-learn numpy seaborn matplotlib Scrapy beautifulsoup4 requests pyprind selenium

     |████████████████████████████████| 254 kB 245 kB/s eta 0:00:01
     |████████████████████████████████| 115 kB 220 kB/s eta 0:00:01
  Using cached requests-2.25.1-py2.py3-none-any.whl (61 kB)
     |████████████████████████████████| 3.1 MB 271 kB/s eta 0:00:01
     |████████████████████████████████| 54 kB 248 kB/s eta 0:00:01
     |████████████████████████████████| 208 kB 294 kB/s eta 0:00:01
     |████████████████████████████████| 2.0 MB 277 kB/s eta 0:00:01
     |████████████████████████████████| 47 kB 289 kB/s eta 0:00:01
  Using cached lxml-4.6.3-cp39-cp39-macosx_10_9_x86_64.whl (4.6 MB)
     |████████████████████████████████| 65 kB 175 kB/s eta 0:00:01
     |████████████████████████████████| 3.2 MB 380 kB/s eta 0:00:01
  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
  Using cached pyasn1-0.4.8-py2.py3-none-any.whl (77 kB)
     |████████████████████████████████| 74 kB 284 kB/s eta 0:00:01
     |████████████████████████████████| 59 kB 430 kB/s eta 0:00:01
     |

# Testing BS4

In [56]:
from bs4 import BeautifulSoup
import requests
import lxml
import time

In [13]:
# Fetching MonstorIndia Sitemap

SITE = 'https://www.monsterindia.com/jdsitemaps/activejobs-in.xml'
my_headers = {
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 ' + "(KHTML, like Gecko) Chrome/61.0.3163.100Safari/537.36"
}

content = requests.get(SITE, headers=my_headers)

<Response [200]>

## Crawling Monstering India Website - Search Page

+ Crawling is done using Selenium with firefox driver, in headless more, which runs in background without disturbing the foreground processes

+ Steps Followed:
    + Form the Search URL, find the parameters required to paginate programmatically.
        + This is done by modifying `?start=`, which stands for start after these N records
    + Scroll twice with a gap of 1 second, for dynamic page load via javascript to finish
    + download the web page


In [157]:
from selenium import webdriver
import pandas as pd
import os
import random

In [159]:
def scroll_down(driver):
    driver.set_window_size(1440, 875)
    driver.execute_script("window.scrollTo(0,397)")
    time.sleep(1)
    driver.execute_script("window.scrollTo(0,1410)")
    time.sleep(1)
    driver.execute_script("window.scrollTo(0,24695)")

def get_parameterized_url(start_after):
    return f'https://www.monsterindia.com/srp/results?start={start_after}&sort=2&limit=100&query=it%2Ccomputers%2Csoftware&salaryRanges=300000~600000,0~300000,1000000~1500000,2500000~*,600000~1000000,1500000~2500000&filter=true'


def download_page(page_src,name):
    os.environ['MOZ_HEADLESS'] = '1'
    driver = webdriver.Firefox(executable_path='/Users/admin/Projects/Msc/Data Mining/Scrapping/geckodriver')
    driver.get(page_src)
    scroll_down(driver)
    with open(f"../raw/{name}.html", "w") as f:
        f.write(driver.page_source)
    driver.close()

In [162]:
TOTAL_POSTINGS = 6000
TOTAL_PAGES = TOTAL_POSTINGS // 100

bar = pyprind.ProgBar(TOTAL_PAGES)
for i in range(TOTAL_PAGES):
    download_page(get_parameterized_url(i), f"monster-india-filtered-by-salary/monster-india-search-page-{i}")
    # after every 10 requests wait for 4 seconds
    if(i % 10 == 0):
        time.sleep(random.randint(0,4))
    bar.update()
print(bar)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:13:05


Title: 
  Started: 06/01/2021 09:30:22
  Finished: 06/01/2021 09:43:28
  Total time elapsed: 00:13:05


## Parsing Search Pages

In [178]:
all_jobs_listings = os.listdir('../raw/monster-india-filtered-by-salary')

In [307]:
def get_job_posting_date(footer_node):
    return {'timestamp' :footer_node.find(class_='posted').text.strip()}

def get_job_title_info(node):
    title_node = node.find(class_='job-tittle')
    job_title = title_node.find('h3').text.strip()
    location = title_node.find(class_='loc').text.strip()
    company_name = title_node.find(class_='company-name').text.strip()
    package = title_node.find(class_='loc salarySnb').text.strip()
    experience = title_node.find(class_='exp').text.strip()
    
    return {'location': location, 'job_title': job_title, 
            'company_name': company_name, 'package': package, 
            'experience': experience}

def get_job_desc(node):
    job_desc_node = node.find(class_='job-descrip').text.strip()
    return {'job_description': job_desc_node}

def get_job_skills(node):
    skills_node = node.find(class_='descrip-skills')
    skills = []
    if skills_node:
        skills_nodes = skills_node.find_all('a')
        for s_node in skills_nodes:
            skills.append(s_node.text)
    remove_special_char = map(lambda x: x.replace('\n','').replace(',','').strip(), skills)
    filtered_skills = filter(lambda x: len(x) > 1, remove_special_char)
    skills_string = ','.join(filtered_skills)
    return {'skills': skills_string}

In [308]:
get_job_skills(job_card)

{'skills': '.Net Developer,Dotnet.Net. Net,Angular,Entity Framework,Mvc'}

In [312]:
all_job_posts = list()
bar = pyprind.ProgBar(len(all_jobs_listings))
for search_page in all_jobs_listings:
    bar.update()
    try:
        path = '../raw/monster-india-filtered-by-salary' + '/' + search_page
        soup = BeautifulSoup(open(path), 'html.parser') 
        # Notice the strucutre of each jobs card, it has the className job-apply-card.
        all_job_cards = soup.find_all(class_='job-apply-card')

        for job_card in all_job_cards:
            try:
                # card-body and card-footer are the child elements containing relavant content
                card_body_node = job_card.find(class_='card-body')
                card_footer = job_card.find(class_='card-footer')

                data = dict()
                posted_on = get_job_posting_date(card_footer)
                title_info = get_job_title_info(card_body_node)
                job_desc = get_job_desc(card_body_node)
                job_skills = get_job_skills(card_body_node)

                data.update(posted_on)
                data.update(title_info)
                data.update(job_desc)
                data.update(job_skills)

                all_job_posts.append(data)
            except Exception as e:
                print('Failed to parse node' + e)
                
    except Exception as e:
        print(e)



0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:50


In [314]:
df = pd.DataFrame(all_job_posts)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16800 entries, 0 to 16799
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   timestamp        16800 non-null  object
 1   location         16800 non-null  object
 2   job_title        16800 non-null  object
 3   company_name     16800 non-null  object
 4   package          16800 non-null  object
 5   experience       16800 non-null  object
 6   job_description  16800 non-null  object
 7   skills           16800 non-null  object
dtypes: object(8)
memory usage: 1.0+ MB


In [319]:
df.package.unique()

array(['60,000-3,00,000 INR Per Annum', '50,000-2,00,000 INR Per Annum',
       '50,000-3,00,000 INR Per Annum', ...,
       '4,20,000-8,80,000 INR Per Annum',
       '3,00,000-12,00,000 INR Per Annum',
       '28,00,000-35,00,000 INR Per Annum'], dtype=object)

In [328]:
df[df.skills.str.contains('android')]

,timestamp,location,job_title,company_name,package,experience,job_description,skills
1735,Posted: a month ago,Chennai,App Developer,Indytek Solutions,"1,00,000-2,50,000 INR Per Annum",1-3 Years,We are look for App developer with minimum 1 Y...,"ios developer,android developer,React Native"
2374,Posted: 11 days ago,"Chennai, India",Android Developer,ANI Calls India Private Limited,"9,00,000-14,00,000 INR Per Annum",5-9 Years,Job Description :\n\n• 5+ years of Mobile Andr...,android developer
2525,Posted: a month ago,Pune,Android Developer,Populus Flyers Private Limited,0-0 INR Per Annum,2-5 Years,We are looking for Android Developer with mini...,"android developer,Android SDK,Software Enginee..."
7039,Posted: 30+ days ago,Noida,Senior Android Developer – 2-4yrs Exp – 3-5L –...,Angel & Genie,"3,00,000-5,00,000 INR Per Annum",2-4 Years,Job Description :\n \n \n \n \n \n \n \n \n \n...,"android developer,Android UI design,C++,Java,M..."
7630,Posted: 9 days ago,Lucknow,iOS developer @work from home,Aashish Krishna Solution,"2,40,000-3,60,000 INR Per Annum",0-1 Years,hiring for iOS developer,"ios developer,android developer,Mobile Developer"
7875,Posted: 25 days ago,Kozhikode / Calicut,Android Developer,Mejora Jobs And Education,"15,000-25,000 INR Per Annum",1-2 Years,Job Description :\n\n\nAndroid Developer\nLoca...,android developer
8011,Posted: 30+ days ago,Indore,Urgent looking for Sr. Android Developer /#And...,Tekit Solution,"3,00,000-4,09,999 INR Per Annum",2-4 Years,We are looking for an Android Developer who po...,"Android native,Android/iOS,android developer,R..."
8581,Posted: 22 days ago,"Hyderabad / Secunderabad, Jaipur",Senior Android Developer,Aarika Hr Professionals Private Limited,"9,40,000-17,50,000 INR Per Annum",3-8 Years,"Excellent knowledge in Open Source, JAVA and A...","android developer,Dragger,Broadcast Receivers,..."
8582,Posted: 22 days ago,Jaipur,Technical Architect - Mobile (Android),Aarika Hr Professionals Private Limited,"25,00,000-27,00,000 INR Per Annum",7-12 Years,"Proven ability to design, optimize and integra...","mobile application architect,Java Architect,J2..."
8863,Posted: 14 days ago,"Noida, India",Android Developer,ANI Calls India Private Limited,"9,00,000-14,00,000 INR Per Annum",5-9 Years,Job Description :\n\n• 8-10+ years of Software...,android developer


In [321]:
df.sample(10)

,timestamp,location,job_title,company_name,package,experience,job_description,skills
1098,Posted: 30+ days ago,"Bengaluru / Bangalore, Chennai",ReactJS Full Stack Developer | 4-6 LPA | 3+ Years,Disha Recruitment Services,"4,00,000-6,00,000 INR Per Annum",3-7 Years,"Key Skills 3+ Years : ReactJS, PHP / CSS, MySQ...","Reactjs,Php,CSS,MySQL,Mangodb,SQL Server,Java,..."
865,Posted: 30+ days ago,"Mumbai, Mumbai City, Navi Mumbai, Thane",required back office executive/computer operat...,Company Name Confidential,"1,10,000-3,20,000 INR Per Annum",0-5 Years,Knowledge of Ms Office.\n Good typing speed.\...,"Back Office,Back Office Executive,Office Assis..."
5639,Posted: 30+ days ago,"Bengaluru / Bangalore, India",POS Manual tester,ANI Calls India Private Limited,"8,00,000-14,00,000 INR Per Annum",8-10 Years,Job Description :\n\n• Oracle Point of Sales S...,POS Manual tester
6204,Posted: 30+ days ago,Bengaluru / Bangalore,Dot Net Developer@Bangalore,iQuest Management Consultants Private Limited,"10,00,000-15,00,000 INR Per Annum",5-10 Years,Roles and Responsibilities\nRequired skills: C...,"C#,core,.NET,MongoDB,Net"
9049,Posted: 30+ days ago,Bengaluru / Bangalore,Embedded Networking,2COMS Consulting Private Limited,"10,00,000-20,00,000 INR Per Annum",6-11 Years,Responsibilities\n\n * Design and implement s...,"Rtos,Firmware,Kernel"
12437,Posted: 30+ days ago,Chennai,DATA Center Operation || Chennai,2COMS Consulting Private Limited,"50,000-2,00,000 INR Per Annum",1-6 Years,DATA CENTRE OPERATION :\nThe challenges of Ent...,"Troubleshooting,Laptop,Desktop,Desktop Support..."
8604,Posted: 11 days ago,"Chennai, India",FINRA,ANI Calls India Private Limited,"8,00,000-14,00,000 INR Per Annum",6-9 Years,Job Description :\n\n• Provides information on...,finra
2889,Posted: 30+ days ago,Bengaluru / Bangalore,Hiring For an International voice / technical ...,Job Shop,"1,75,000-4,25,000 INR Per Annum",0-5 Years,Greetings from Job Shop!\nWORK FROM HOME WITH ...,"Csa,International Call Center,US Shift,Night S..."
13037,Posted: 12 days ago,"Kolkata, Asansol","Diploma,degree,B.tech ,B.E,diploma hiring in ...",Freelancer Ganesh Popat Sontakke,"2,70,000-12,90,000 INR Per Annum",0-5 Years,"Diploma,Degree,B.E,B.TECH,Mechanical Engineer,...","Diploma,Degree,B.E,B.TECH,Mechanical Engineer,..."
5494,Posted: 14 days ago,"Chennai, India",Java Microservices,ANI Calls India Private Limited,"9,00,000-14,00,000 INR Per Annum",6-9 Years,Job Description :\n\n• Have a powerful Web/Res...,Java Microservices


In [315]:
df.to_csv('monster-india-jobs.csv')

### Sources

+ Headless Firefox : https://stackoverflow.com/questions/46753393/how-to-make-firefox-headless-programmatically-in-selenium-with-python
